In [1]:
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from datetime import datetime 
import datetime
import collections
from collections import Counter
from collections import OrderedDict
import glob
import time
import os
from joblib import Parallel, delayed
from multiprocessing import Pool
import seaborn as sns
%matplotlib inline

In [2]:
import sklearn.datasets

# 数値計算に使うライブラリ
import scipy as sp
from scipy import stats as st

# グラフを描画するライブラリ
sns.set()

# 統計モデルを推定するライブラリ
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.anova as anova # 分散分析するライブラリ
from statsmodels.stats.multicomp import pairwise_tukeyhsd # Tukeyの多重比較するライブラリ

In [3]:
import scipy.stats as stats
from datetime import datetime as dt
from sklearn.ensemble import  RandomForestRegressor as RFR
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV
import optuna.integration.lightgbm as lgb_o
import lightgbm as lgb
import xgboost as xgb


import optuna.integration.lightgbm as lgb

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

## 全CSVファイルを結合

In [4]:
llist = []
allfiles = glob.glob('Data/*.csv')
    
for file in allfiles:
    llist.append(pd.read_csv(file,header=None))
df = pd.concat(llist, sort=False)
df = df.drop(index=0)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [164]:
df

,0,1,2,3,4,5,6
1,2020/01/20,23,47:沖縄県,平日,都営地下鉄大江戸線,38:新宿西口駅,274
2,2020/01/20,22,47:沖縄県,平日,都営地下鉄大江戸線,38:新宿西口駅,123
3,2020/01/20,21,47:沖縄県,平日,都営地下鉄大江戸線,38:新宿西口駅,400
4,2020/01/20,20,47:沖縄県,平日,都営地下鉄大江戸線,38:新宿西口駅,463
5,2020/01/20,19,47:沖縄県,平日,都営地下鉄大江戸線,38:新宿西口駅,358
...,...,...,...,...,...,...,...
715800,2020/12/31,14,01:北海道,平日,都営地下鉄三田線,27:目黒駅,19
715801,2020/12/31,0,01:北海道,平日,都営地下鉄三田線,25:白金高輪駅,11
715802,2020/12/31,17,01:北海道,平日,都営地下鉄三田線,13:巣鴨駅,7
715803,2020/12/31,19,01:北海道,平日,都営地下鉄三田線,12:西巣鴨駅,2


## 日付・時間を整理

In [8]:
df = df.rename(columns={0:'Date', 1:'Time', 2:'From', 3:'Weekday/Holidays', 4:'Train Line', 5:'Station', 6:'Population'})
df = df.sort_values(['Date', 'Time'])

## 朝の時間帯（6時〜9時）だけを抽出（2020/2〜2021/6）

In [9]:
ndf = df.copy()
ndf['Date'] = pd.to_datetime(ndf['Date'])
ndf['Time'] = ndf['Time'].astype(int)
ndf['Population'] = ndf['Population'].astype(int)
ndf = ndf[(ndf['Time'] >= 6) & (ndf['Time'] <= 9)]
ndf = ndf[ndf['Date'] >= datetime.datetime(year=2020, month=2, day=1)]
display(ndf)

,Date,Time,From,Weekday/Holidays,Train Line,Station,Population
182710,2020-02-01,6,47:沖縄県,休日,都営地下鉄大江戸線,37:東新宿駅,181
182754,2020-02-01,6,47:沖縄県,休日,都営地下鉄大江戸線,32:春日駅,266
182784,2020-02-01,6,47:沖縄県,休日,都営地下鉄大江戸線,27:両国駅,230
182795,2020-02-01,6,47:沖縄県,休日,都営地下鉄大江戸線,24:門前仲町駅,205
182824,2020-02-01,6,47:沖縄県,休日,都営地下鉄大江戸線,20:汐留駅,311
...,...,...,...,...,...,...,...
318914,2021-06-25,9,08:茨城県,平日,都営地下鉄三田線,23:芝公園駅,9
318928,2021-06-25,9,08:茨城県,平日,都営地下鉄三田線,22:御成門駅,259
318940,2021-06-25,9,08:茨城県,平日,都営地下鉄三田線,20:日比谷駅,136
318955,2021-06-25,9,08:茨城県,平日,都営地下鉄三田線,17:水道橋駅,286


In [28]:
nndf = ndf.drop(['From', 'Time','Weekday/Holidays', 'Train Line'], axis=1)
df1 = nndf.groupby('Date').sum()['Population']
df2 = ndf.groupby(['Date', 'Station']).sum()
df_newconcat = pd.merge(df2, df1, on='Date', how='left')

## 駅利用者数順にする

In [51]:
nndf = ndf.drop(['From', 'Time','Weekday/Holidays', 'Train Line'], axis=1)
df1 = nndf.groupby('Date').sum()['Population']
df2 = ndf.groupby(['Date', 'Station']).sum()
df_newconcat = pd.merge(df2, df1, on='Date', how='left')
df2_index = list(df2.index)
df2_li=[]
for i in range(len(df2_index)):
    df2_li.append(df2_index[i][1])
df_newconcat['Station'] = df2_li
df_newconcat['rate'] = df_newconcat['Population_x']*100/df_newconcat['Population_y']
df_newconcat = df_newconcat.drop(['Time','Population_x', 'Population_y'], axis=1)

rate_df = df_newconcat.groupby('Station').sum()
rate_df = rate_df.sort_values('rate', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
rate_df

,rate
Station,
21:新宿駅,1273.211204
12:新宿駅,1166.257437
38:新宿西口駅,1149.984977
11:都庁前駅,944.948795
39:都庁前駅,944.948795
11:新橋駅,858.400952
8:日本橋駅,856.976256
19:大手町駅,813.874134
21:内幸町駅,809.337399


## ここまで

In [8]:
## 不要な列削除
ini_df = ndf.drop(['From', 'Time','Weekday/Holidays', 'Train Line', 'Station'], axis=1)
ini_df = ini_df.groupby('Date').sum()
ini_df = ini_df.sort_values('Date', ascending=False)
ini_df

,Population
Date,
2021-06-25,20422985
2021-06-24,20729214
2021-06-23,20568891
2021-06-22,20762609
2021-06-21,20785624
...,...
2020-02-05,26778079
2020-02-04,26879466
2020-02-03,26776675


## 教師データに合わせて成形

In [9]:
ini_df = ini_df.reset_index()
#np.set_printoptions(threshold=np.inf)
#pd.set_option('display.max_rows', 500)
s = 5
t = 6
for i in range(len(ini_df)):
    if i == s:
        ini_df = ini_df.drop(s)
        ini_df = ini_df.drop(t)
        s = s+7
        t = t+7
        
print(ini_df)

          Date  Population
0   2021-06-25    20422985
1   2021-06-24    20729214
2   2021-06-23    20568891
3   2021-06-22    20762609
4   2021-06-21    20785624
..         ...         ...
504 2020-02-07    26869122
505 2020-02-06    26888857
506 2020-02-05    26778079
507 2020-02-04    26879466
508 2020-02-03    26776675

[365 rows x 2 columns]


## 5日おきにデータをとる

In [10]:
# ini_df['Date'] = pd.to_datetime(ini_df['Date'])
# df.groupby('Population').count()
ini_df = ini_df.reset_index()
pre_num=0
app_list=[]
j=4
for i in range(len(ini_df['Population'])):
    if i <= j:
        pre_num = pre_num + ini_df['Population'][i]
    else:
        j = j + 5
        app_list.append(pre_num)
        pre_num = ini_df['Population'][i]
print(app_list)

[103269323, 100936661, 100460873, 100712619, 99666299, 100148972, 101518958, 68250476, 88346375, 104606322, 105220648, 105229810, 105943569, 102359869, 99563436, 99614504, 99169475, 90147280, 97044186, 88622929, 95768290, 94760738, 94644698, 87519625, 97182402, 59442869, 105080964, 106241340, 106750118, 107271868, 97630331, 107363679, 108810609, 99017274, 108118458, 107048947, 106834501, 107467319, 107280471, 84407146, 104291949, 104459748, 103851708, 101811891, 100331708, 74835617, 100248487, 102431241, 83687988, 103254386, 104888848, 105574963, 101866733, 100018158, 97776699, 97718777, 85516865, 80681984, 79542978, 60959696, 69129475, 76687760, 79983328, 94726897, 109208550, 118677506, 109156395, 119724079, 121137837, 113870799, 130814878, 117497017]


## 教師データの各行に対する合計値を算出

In [14]:
teach_df = pd.read_csv('teach.csv',header=None)
teach_df = teach_df.drop(index=0)
teach_df = teach_df.drop(index=1)
teach_df = teach_df.rename(columns={0:'Year', 1:'Date', 2:'Number of Person(1)', 3:'Number of Person(2)', 4:'Number of Person(3)'})
teach_df['Number of Person(1)'] = teach_df['Number of Person(1)'].astype(int)
teach_df['Number of Person(2)'] = teach_df['Number of Person(2)'].astype(int)
teach_df['Number of Person(3)'] = teach_df['Number of Person(3)'].astype(int)
teach_df = pd.concat([teach_df,pd.DataFrame(teach_df.sum(axis=1),columns=['Total'])],axis=1)
teach_df

/var/folders/g3/mlwq72x91z37blvqkhp09ksw0000gr/T/ipykernel_68678/4133738615.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  teach_df = pd.concat([teach_df,pd.DataFrame(teach_df.sum(axis=1),columns=['Total'])],axis=1)


,Year,Date,Number of Person(1),Number of Person(2),Number of Person(3),Total
2,2021,6月21日~6月25日,219053,369857,406336,995246
3,2021,6月14日~6月18日,214606,359320,394999,968925
4,2021,6月7日~6月11日,213213,355557,383042,951812
5,2021,5月31日~6月4日,213971,358621,384777,957369
6,2021,5月24日~5月28日,212407,353622,379883,945912
...,...,...,...,...,...,...
70,2020,3月2日~3月6日,245463,417379,556567,1219409
71,2020,2月25日~2月28日,251791,486673,574223,1312687
72,2020,2月17日~2月21日,243435,515488,576453,1335376
73,2020,2月10日~2月14日,241969,521778,573169,1336916


## 前処理データと教師データを結合

In [15]:
teach_df = teach_df.drop(['Number of Person(1)', 'Number of Person(2)', 'Number of Person(3)'], axis=1)
teach_df = teach_df.reset_index()
teach_df = teach_df[:-1]
teach_df = pd.concat([teach_df,pd.DataFrame(app_list,columns=['Train_Total'])],axis=1)
teach_df

,index,Year,Date,Total,Train_Total
0,2,2021,6月21日~6月25日,995246,103269323
1,3,2021,6月14日~6月18日,968925,100936661
2,4,2021,6月7日~6月11日,951812,100460873
3,5,2021,5月31日~6月4日,957369,100712619
4,6,2021,5月24日~5月28日,945912,99666299
...,...,...,...,...,...
67,69,2020,3月9日~3月13日,1190078,119724079
68,70,2020,3月2日~3月6日,1219409,121137837
69,71,2020,2月25日~2月28日,1312687,113870799
70,72,2020,2月17日~2月21日,1335376,130814878


## 天気情報

In [32]:
# pd.set_option('display.max_rows', 500)
df_wea = pd.read_csv('weather.csv')
df_wea = df_wea.rename(columns={'年月日':'Date', '平均気温(℃)':'Average temperature', '最高気温(℃)':'Maximum　temperature', '最低気温(℃)':'Minimum　temperature', '降水量の合計(mm)':'Total precipitation'})
df_wea = df_wea.drop(index=df_wea.index[[511,512,513,514,515]],columns=["平均雲量(10分比)","平均雲量(10分比).1","平均雲量(10分比).2","平均気温(℃).1","平均気温(℃).2","最高気温(℃).1", "最高気温(℃).2", "最低気温(℃).1", "最低気温(℃).2", "降水量の合計(mm).1","降水量の合計(mm).2", "降水量の合計(mm).3"])

df_wea['Date'] = pd.to_datetime(df_wea['Date'])
df_wea = df_wea.sort_values('Date', ascending=False)


# 天気データを教師データに合わせて成形
df_wea = df_wea.reset_index()

u = 5
v = 6
for i in range(len(df_wea)):
    if i == u:
        df_wea = df_wea.drop(u)
        df_wea = df_wea.drop(v)
        u = u+7
        v = v+7

# 5日間の平均を出す
df_wea = df_wea.reset_index()
ave_=0
max_= 0
min_=0
pre_=0
ave_1=0
max_1= 0
min_1=0
pre_1=0
ave_li=[]
max_li=[]
min_li=[]
pre_li=[]
j=4
for i in range(len(df_wea['Average temperature'])):
    if i <= j:
        ave_ = ave_ + df_wea['Average temperature'][i]
        max_ = max_ + df_wea['Maximum　temperature'][i]
        min_ = min_ + df_wea['Minimum　temperature'][i]
        pre_ = pre_ + df_wea['Total precipitation'][i]
    else:
        j = j + 5
        ave_1 = round(ave_/5, 2)
        ave_li.append(ave_1)
        ave_ = df_wea['Average temperature'][i]
        
        max_1 = round(max_/5, 2)
        max_li.append(max_1)
        max_ = df_wea['Maximum　temperature'][i]
        
        min_1 = round(min_/5, 2)
        min_li.append(min_1)
        min_ = df_wea['Minimum　temperature'][i]
        
        pre_1 = round(pre_/5, 2)
        pre_li.append(pre_1)
        pre_ = df_wea['Total precipitation'][i]


## 人流データと天気情報を結合

In [35]:
last_df = pd.concat([teach_df,pd.DataFrame(ave_li,columns=['Average temperature'])],axis=1)
last_df = pd.concat([last_df,pd.DataFrame(max_li,columns=['Maximum　temperature'])],axis=1)
last_df = pd.concat([last_df,pd.DataFrame(min_li,columns=['Minimum　temperature'])],axis=1)
last_df = pd.concat([last_df,pd.DataFrame(pre_li,columns=['Total precipitation'])],axis=1)
last_df = last_df.drop(['index'], axis=1)
last_df

,Year,Date,Total,Train_Total,Average temperature,Maximum temperature,Minimum temperature,Total precipitation
0,2021,6月21日~6月25日,995246,103269323,22.82,27.36,19.82,4.1
1,2021,6月14日~6月18日,968925,100936661,22.24,26.44,19.16,7.7
2,2021,6月7日~6月11日,951812,100460873,24.10,30.46,19.52,0.0
3,2021,5月31日~6月4日,957369,100712619,21.16,25.72,17.58,6.8
4,2021,5月24日~5月28日,945912,99666299,20.60,25.86,16.16,7.1
5,2021,5月17日~5月21日,938993,100148972,20.30,22.74,17.10,3.6
6,2021,5月10日~5月14日,934250,101518958,18.50,22.26,15.34,2.4
7,2021,5月6日~5月7日,887643,68250476,17.74,22.86,13.44,1.5
8,2021,4月26日~4月30日,913931,88346375,16.44,21.32,11.30,8.0
9,2021,4月19日~4月23日,1014770,104606322,16.80,23.64,11.14,0.0


## 学習データとテストデータを分ける

In [122]:
last_df = last_df.drop(['Date'], axis=1)

In [123]:
learn_df = last_df[12:]
test_df = last_df[:12]

## データを標準化

In [124]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(learn_df)
scaler.transform(learn_df)
scaler.transform(test_df)
learn_df_std = pd.DataFrame(scaler.transform(learn_df), columns=learn_df.columns)
test_df_std = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)

## Light GBM

In [129]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'vervose': 0
}

RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2
score_list = []


x_train = learn_df_std[['Train_Total','Average temperature', 'Maximum　temperature','Minimum　temperature', 'Total precipitation']]
y_train = learn_df_std['Total']
x_test = test_df_std[['Train_Total','Average temperature', 'Maximum　temperature','Minimum　temperature', 'Total precipitation']]
y_test = test_df_std['Total']

# # 学習データと評価データを作成
# x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 0:df.shape[1]-1],
#                                                     df.iloc[:, df.shape[1]-1],
#                                                     test_size=TEST_SIZE,
#                                                     random_state=RANDOM_STATE)

# trainのデータセットの3割をモデル学習時のバリデーションデータとして利用する
x_train, x_valid, y_train, y_valid = train_test_split(x_train,
                                                    y_train,
                                                    test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE)

# LightGBMを利用するのに必要なフォーマットに変換
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

# ベストなパラメータ、途中経過を保存する
params = {
    'objective': 'mean_squared_error',
    'metric': 'mae',
    "verbosity": -1,
    "boosting_type": "gbdt",
}

best_params, history = {}, []

# LightGBM学習
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=[lgb_train, lgb_eval],
                early_stopping_rounds=50
               )

best_params = gbm.params
best_params

# LightGBM推論
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)



# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame({'R2': r2_score(true, pred),
                          'MAE': mean_absolute_error(true, pred),
                          'MSE': mean_squared_error(true, pred),
                          'RMSE': np.sqrt(mean_squared_error(true, pred)),
                          'MAPE' : mean_absolute_percentage_error(true, pred)},
                           index = ['scores'])
    return scores
scores = calculate_scores(y_test, y_pred)
score_list.append(scores)
print("="*50)
print(scores)

print(score_list)

[I 2021-09-07 21:22:10,801] A new study created in memory with name: no-name-31319203-f3be-4d16-974a-c6907f032ad4

feature_fraction, val_score: 0.332634:   0%|              | 0/7 [00:00<?, ?it/s][I 2021-09-07 21:22:10,909] Trial 0 finished with value: 0.33263410342517996 and parameters: {'feature_fraction': 0.6}. Best is trial 0 with value: 0.33263410342517996.

feature_fraction, val_score: 0.309903:  29%|#7    | 2/7 [00:00<00:00, 11.53it/s][I 2021-09-07 21:22:10,984] Trial 1 finished with value: 0.3099029256944839 and parameters: {'feature_fraction': 0.7}. Best is trial 1 with value: 0.3099029256944839.

feature_fraction, val_score: 0.309903:  29%|#7    | 2/7 [00:00<00:00, 11.53it/s]

[1]	valid_0's l1: 0.728592	valid_1's l1: 0.446891
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 0.724071	valid_1's l1: 0.463959
[3]	valid_0's l1: 0.667241	valid_1's l1: 0.437074
[4]	valid_0's l1: 0.626314	valid_1's l1: 0.410251
[5]	valid_0's l1: 0.587311	valid_1's l1: 0.387942
[6]	valid_0's l1: 0.563579	valid_1's l1: 0.365719
[7]	valid_0's l1: 0.542863	valid_1's l1: 0.347204
[8]	valid_0's l1: 0.542505	valid_1's l1: 0.357856
[9]	valid_0's l1: 0.530237	valid_1's l1: 0.339373
[10]	valid_0's l1: 0.530218	valid_1's l1: 0.344333
[11]	valid_0's l1: 0.520497	valid_1's l1: 0.332634
[12]	valid_0's l1: 0.52034	valid_1's l1: 0.341598
[13]	valid_0's l1: 0.514711	valid_1's l1: 0.333979
[14]	valid_0's l1: 0.514271	valid_1's l1: 0.344521
[15]	valid_0's l1: 0.50944	valid_1's l1: 0.343861
[16]	valid_0's l1: 0.509501	valid_1's l1: 0.339599
[17]	valid_0's l1: 0.511681	valid_1's l1: 0.336783
[18]	valid_0's l1: 0.513626	valid_1's l1: 0.341615
[19]	valid_0's l1: 0.515527	vali


feature_fraction, val_score: 0.299988:  29%|#7    | 2/7 [00:00<00:00, 11.53it/s][I 2021-09-07 21:22:11,065] Trial 2 finished with value: 0.2999883428586631 and parameters: {'feature_fraction': 0.4}. Best is trial 2 with value: 0.2999883428586631.

feature_fraction, val_score: 0.298589:  57%|###4  | 4/7 [00:00<00:00, 12.21it/s][I 2021-09-07 21:22:11,141] Trial 3 finished with value: 0.29858903916974283 and parameters: {'feature_fraction': 1.0}. Best is trial 3 with value: 0.29858903916974283.

feature_fraction, val_score: 0.298589:  57%|###4  | 4/7 [00:00<00:00, 12.21it/s][I 2021-09-07 21:22:11,194] Trial 4 finished with value: 0.3099029256944839 and parameters: {'feature_fraction': 0.8}. Best is trial 3 with value: 0.29858903916974283.

feature_fraction, val_score: 0.298589:  71%|####2 | 5/7 [00:00<00:00, 12.21it/s]

[33]	valid_0's l1: 0.507798	valid_1's l1: 0.380137
[34]	valid_0's l1: 0.504485	valid_1's l1: 0.382346
[35]	valid_0's l1: 0.504403	valid_1's l1: 0.384136
[36]	valid_0's l1: 0.505755	valid_1's l1: 0.38767
[37]	valid_0's l1: 0.503475	valid_1's l1: 0.38919
[38]	valid_0's l1: 0.503495	valid_1's l1: 0.392279
[39]	valid_0's l1: 0.504452	valid_1's l1: 0.392279
[40]	valid_0's l1: 0.505394	valid_1's l1: 0.393267
[41]	valid_0's l1: 0.505333	valid_1's l1: 0.395246
[42]	valid_0's l1: 0.502802	valid_1's l1: 0.3983
[43]	valid_0's l1: 0.50129	valid_1's l1: 0.401065
[44]	valid_0's l1: 0.501794	valid_1's l1: 0.401065
[45]	valid_0's l1: 0.502562	valid_1's l1: 0.401065
[46]	valid_0's l1: 0.503398	valid_1's l1: 0.402094
[47]	valid_0's l1: 0.503347	valid_1's l1: 0.404537
[48]	valid_0's l1: 0.503302	valid_1's l1: 0.406735
[49]	valid_0's l1: 0.501572	valid_1's l1: 0.409184
[50]	valid_0's l1: 0.501941	valid_1's l1: 0.409184
[51]	valid_0's l1: 0.502119	valid_1's l1: 0.411087
[52]	valid_0's l1: 0.502446	valid_1'


feature_fraction, val_score: 0.298589:  86%|#####1| 6/7 [00:00<00:00, 14.27it/s][I 2021-09-07 21:22:11,253] Trial 5 finished with value: 0.29858903916974283 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 3 with value: 0.29858903916974283.

feature_fraction, val_score: 0.298589: 100%|######| 7/7 [00:00<00:00, 14.16it/s]


...

[12]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[13]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[14]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[15]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[16]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[17]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[18]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[19]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[20]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[21]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[22]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[23]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[24]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[25]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[26]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[27]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[28]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[29]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[30]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[31]	valid_0's l1: 0.789464	val


min_data_in_leaf, val_score: 0.275352:  40%|##4   | 2/5 [00:00<00:00,  7.74it/s][I 2021-09-07 21:22:15,006] Trial 61 finished with value: 0.27535244800444797 and parameters: {'min_child_samples': 5}. Best is trial 61 with value: 0.27535244800444797.

min_data_in_leaf, val_score: 0.275352:  40%|##4   | 2/5 [00:00<00:00,  7.74it/s][I 2021-09-07 21:22:15,045] Trial 62 finished with value: 0.4907380232665067 and parameters: {'min_child_samples': 50}. Best is trial 61 with value: 0.27535244800444797.

min_data_in_leaf, val_score: 0.275352:  60%|###6  | 3/5 [00:00<00:00,  7.74it/s]

[96]	valid_0's l1: 0.100659	valid_1's l1: 0.291739
[97]	valid_0's l1: 0.0995725	valid_1's l1: 0.292315
[98]	valid_0's l1: 0.0979819	valid_1's l1: 0.292955
[99]	valid_0's l1: 0.0976253	valid_1's l1: 0.292946
[100]	valid_0's l1: 0.0969757	valid_1's l1: 0.293485
[101]	valid_0's l1: 0.0951572	valid_1's l1: 0.293478
[102]	valid_0's l1: 0.0945793	valid_1's l1: 0.293982
[103]	valid_0's l1: 0.0929074	valid_1's l1: 0.293973
[104]	valid_0's l1: 0.0924646	valid_1's l1: 0.29445
[105]	valid_0's l1: 0.0914346	valid_1's l1: 0.295259
Early stopping, best iteration is:
[55]	valid_0's l1: 0.178487	valid_1's l1: 0.275352
[1]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[3]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[4]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[5]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[6]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[7]	valid_0's l1: 0.789464	va

[I 2021-09-07 21:22:15,101] Trial 63 finished with value: 0.3495261124718351 and parameters: {'min_child_samples': 10}. Best is trial 61 with value: 0.27535244800444797.

min_data_in_leaf, val_score: 0.275352: 100%|######| 5/5 [00:00<00:00, 12.22it/s]

[1]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[3]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[4]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[5]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[6]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[7]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[8]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[9]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[10]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[11]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[12]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[13]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[14]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[15]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[16]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[17]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[18]	valid_0's l1: 0.789464	valid_1's l1: 0.490738
[19]	valid_0's l1: 0.789464	va

## ランダムフォレスト

In [149]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix

# X_train = learn_df[["ble_num", "final_address"]]
X_train = learn_df_std[['Train_Total','Average temperature', 'Maximum　temperature','Minimum　temperature', 'Total precipitation']]
y_train = learn_df_std['Total']
X_test = test_df_std[['Train_Total','Average temperature', 'Maximum　temperature','Minimum　temperature', 'Total precipitation']] 
y_test = test_df_std['Total']

search_params = {
    'n_estimators'      : [5, 10, 20, 30, 50,100],
    #"max_features": [1,10,100,500,1000,3000,len(all_learn_df[test_rasp_id][0]["ble_num"].tolist())],
    'random_state'      : [2525],
    'n_jobs'            : [1],
    'min_samples_split' : [3, 5, 10, 25, 50, 100],
    'max_depth'         : [3, 5, 10, 25, 50, 100]
}
gsr = GridSearchCV(
    RFR(),
    search_params,
    cv = 3,
    n_jobs = -1,
    verbose=True
)
gsr.fit(X_train, y_train)
y_pred = gsr.predict(X_test)
RFR_MAE = mean_absolute_error(y_test, y_pred)
RFR_MAPE = mean_absolute_percentage_error(y_test, y_pred)


print("RFR")
print("MAE: {:.2f}".format(RFR_MAE))
print("MAPE: {:.2f}".format(RFR_MAPE))

print("RFR finished")

Fitting 3 folds for each of 216 candidates, totalling 648 fits


ValueError: All arrays must be of the same length

## TabNet

In [146]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import torch
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

X_train = np.array(learn_df_std['Train_Total']).reshape(-1, 1)
y_train = np.array(learn_df_std['Total']).reshape(-1, 1)
X_test = np.array(test_df_std['Train_Total']).reshape(-1, 1)
y_test = np.array(test_df_std['Total']).reshape(-1, 1)

clf = TabNetRegressor(optimizer_fn=torch.optim.Adam,optimizer_params=dict(lr=0.5))  #TabNetRegressor()
clf.fit(X_train, y_train, loss_fn=torch.nn.functional.l1_loss, max_epochs=500)

Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.8643  |  0:00:00s
epoch 1  | loss: 6.03329 |  0:00:00s
epoch 2  | loss: 1.2933  |  0:00:00s
epoch 3  | loss: 1.21724 |  0:00:00s
epoch 4  | loss: 0.71761 |  0:00:00s
epoch 5  | loss: 0.46548 |  0:00:00s
epoch 6  | loss: 0.41586 |  0:00:00s
epoch 7  | loss: 0.34912 |  0:00:00s
epoch 8  | loss: 0.34206 |  0:00:00s
epoch 9  | loss: 0.50153 |  0:00:00s
epoch 10 | loss: 0.48226 |  0:00:00s
epoch 11 | loss: 0.38489 |  0:00:00s
epoch 12 | loss: 0.63676 |  0:00:00s
epoch 13 | loss: 0.51594 |  0:00:00s
epoch 14 | loss: 0.42366 |  0:00:00s
epoch 15 | loss: 0.45277 |  0:00:00s
epoch 16 | loss: 0.42    |  0:00:00s
epoch 17 | loss: 0.40148 |  0:00:00s
epoch 18 | loss: 0.39859 |  0:00:00s
epoch 19 | loss: 0.39097 |  0:00:00s
epoch 20 | loss: 0.35349 |  0:00:00s
epoch 21 | loss: 0.35095 |  0:00:00s
epoch 22 | loss: 0.3217  |  0:00:00s
epoch 23 | loss: 0.35839 |  0:00:00s
epoch 24 | loss: 0.29

epoch 222| loss: 0.21555 |  0:00:04s
epoch 223| loss: 0.20846 |  0:00:04s
epoch 224| loss: 0.22666 |  0:00:04s
epoch 225| loss: 0.20443 |  0:00:04s
epoch 226| loss: 0.22548 |  0:00:04s
epoch 227| loss: 0.19076 |  0:00:04s
epoch 228| loss: 0.24916 |  0:00:04s
epoch 229| loss: 0.24496 |  0:00:04s
epoch 230| loss: 0.19347 |  0:00:04s
epoch 231| loss: 0.23339 |  0:00:04s
epoch 232| loss: 0.21707 |  0:00:04s
epoch 233| loss: 0.18604 |  0:00:04s
epoch 234| loss: 0.21701 |  0:00:04s
epoch 235| loss: 0.18974 |  0:00:04s
epoch 236| loss: 0.1868  |  0:00:04s
epoch 237| loss: 0.20791 |  0:00:04s
epoch 238| loss: 0.18022 |  0:00:04s
epoch 239| loss: 0.18869 |  0:00:04s
epoch 240| loss: 0.1807  |  0:00:05s
epoch 241| loss: 0.17353 |  0:00:05s
epoch 242| loss: 0.17771 |  0:00:05s
epoch 243| loss: 0.18919 |  0:00:05s
epoch 244| loss: 0.18142 |  0:00:05s
epoch 245| loss: 0.18716 |  0:00:05s
epoch 246| loss: 0.17427 |  0:00:05s
epoch 247| loss: 0.18306 |  0:00:05s
epoch 248| loss: 0.18643 |  0:00:05s
e

epoch 453| loss: 0.16407 |  0:00:09s
epoch 454| loss: 0.1824  |  0:00:09s
epoch 455| loss: 0.18506 |  0:00:09s
epoch 456| loss: 0.14796 |  0:00:09s
epoch 457| loss: 0.19048 |  0:00:09s
epoch 458| loss: 0.18152 |  0:00:09s
epoch 459| loss: 0.14952 |  0:00:09s
epoch 460| loss: 0.19009 |  0:00:09s
epoch 461| loss: 0.18722 |  0:00:09s
epoch 462| loss: 0.17089 |  0:00:09s
epoch 463| loss: 0.15832 |  0:00:09s
epoch 464| loss: 0.16746 |  0:00:09s
epoch 465| loss: 0.15465 |  0:00:09s
epoch 466| loss: 0.15813 |  0:00:09s
epoch 467| loss: 0.16574 |  0:00:09s
epoch 468| loss: 0.15358 |  0:00:09s
epoch 469| loss: 0.17391 |  0:00:09s
epoch 470| loss: 0.16805 |  0:00:09s
epoch 471| loss: 0.16413 |  0:00:09s
epoch 472| loss: 0.16867 |  0:00:09s
epoch 473| loss: 0.16699 |  0:00:09s
epoch 474| loss: 0.1547  |  0:00:09s
epoch 475| loss: 0.16722 |  0:00:09s
epoch 476| loss: 0.14713 |  0:00:09s
epoch 477| loss: 0.1468  |  0:00:09s
epoch 478| loss: 0.15135 |  0:00:09s
epoch 479| loss: 0.14691 |  0:00:09s
e

In [147]:
clf.predict(X_test)

array([[ 0.46348372],
       [ 0.10670522],
       [ 0.05120742],
       [ 0.07974941],
       [-0.02303985],
       [ 0.01877502],
       [ 0.18298042],
       [-2.1085935 ],
       [-0.13549964],
       [ 0.6079467 ],
       [ 0.64919984],
       [ 0.64983326]], dtype=float32)

In [ ]:
##　

In [148]:
from sklearn.metrics import mean_absolute_error
print('MAE : ', mean_absolute_error(y_test, clf.predict(X_test)))
print('MAPE : ', mean_absolute_percentage_error(y_test, clf.predict(X_test)))

MAE :  0.26440947764658146
MAPE :  4.665169313125471
